In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import datetime as dt
import time

from calitp import get_engine
from calitp.tables import tbls

engine = get_engine()
connection = engine.connect()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/ahsc_grant/'

import gcsfs
fs = gcsfs.GCSFileSystem()

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


# Collating Big Stop Table

In [3]:
# read in and concatenate all geoparquets
fs_list = fs.ls(f"{GCS_FILE_PATH}tool_data/")

fs_list[1:]

['calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_10.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_101.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_102.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_103.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_106.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_107.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_108.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_11.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_110.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_112.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_116.parquet',
 'calitp-analytics-data/data-analyses/ahsc_gr

In [4]:
filelist = []
for f in fs_list[1:]:
    try: 
        test_gpqt = gpd.read_parquet(f"gs://{f}")
        filelist.append(f)
    except:
        print(f"error on {f.split('tool_data/')}")
        
filelist

error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_116.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_118.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_186.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_199.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_21.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_212.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_213.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_216.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_23.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_254.parquet']
error on ['calitp-analytics-data/data-analyses/ahsc_grant/', 'trips_perstop_277.parquet']
error on ['c

['calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_10.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_101.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_102.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_103.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_106.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_107.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_108.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_11.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_110.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_112.parquet',
 'calitp-analytics-data/data-analyses/ahsc_grant/tool_data/trips_perstop_117.parquet',
 'calitp-analytics-data/data-analyses/ahsc_gr

In [6]:
gdf_wkd = (pd.concat(
        gpd.read_parquet(f"gs://{f}")
        for f in filelist
        )
           >> filter(_.daytype=="Weekday")
          )

Note: Running into memory issues adding spatial weights matrix. Proceeding without spatially-lagged factors for now.

In [3]:
# put coefficients into arrays - variable order from spatial_regression_exploration_kmk
# in future, save out coefficients somewhere 
import numpy as np

wkd_coeff = np.array([0.0200865,-0.1610594,0.0001214,-0.0000173,0.0224169,-0.0152673,-0.0505976,-0.0423512,0.0111763])
sat_coeff = np.array([0.0262958,-0.1424400,0.0001344,-0.0000186,0.0256008,-0.0169793,-0.0408743,-0.0419725,0.0126354])
sun_coeff = np.array([0.0263988,-0.1082477,0.0001477,-0.0000202,0.0209053,-0.0145447,-0.0449611,-0.0502937, 0.0132250])

In [ ]:
# multiply by coefficients



In [ ]:
#create stop-specific ridership factor, baseline ridership